In [1]:
from IPython.display import display
from IPython.core.display import HTML 
import IPython.core.display as di # Example: di.display_html('<h3>%s:</h3>' % str, raw=True)

# This line will hide code by default when the notebook is exported as HTML
di.display_html('<script>jQuery(function() {if (jQuery("body.notebook_app").length == 0) { jQuery(".input_area").toggle(); jQuery(".prompt").toggle();}});</script>', raw=True)

# This line will add a button to toggle visibility of code blocks, for use with the HTML export version
di.display_html('''<button onclick="jQuery('.input_area').toggle(); jQuery('.prompt').toggle();">Toggle code</button>''', raw=True)

import pandas as pd
import numpy as np

pd.set_option('mode.chained_assignment',None)

Toggle code

In [2]:
from sklearn.metrics import confusion_matrix 
from sklearn.metrics import accuracy_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier

In [3]:
from sklearn.cross_validation import train_test_split

In [4]:
from sklearn.cross_validation import cross_val_score

In [5]:
data_df = pd.read_csv('spotify_top_200_clustered.csv')

In [6]:
data_df.head()

,track_id,Artist,Track.Name,top.rank,total.plays,average.daily.plays,days.in.chart,artist_id,followers,genres,...,speechiness,acousticness,instrumentalness,danceability,key,duration_ms,loudness,mode,valence,cluster
0,003eoIwxETJujVWmNFMoZy,Alessia Cara,Growing Pains,91,9809956,700711.142857,14,2wUjUUtkb5lvLKcGKsKqsR,4431425,"[u'canadian contemporary r&b', u'dance pop', u...",...,0.7330,0.0822,0.000000,0.353,1,193680,-6.276,0,0.437,1
1,00B7TZ0Xawar6NZ00JFomN,Cardi B,Best Life (feat. Chance The Rapper),61,11882167,848726.214286,14,4kYSro6naA4h99UJvo89HB,5927731,"[u'pop', u'rap']",...,0.5530,0.2870,0.000000,0.620,9,284856,-7.438,1,0.665,1
2,00EPIEnX1JFjff8sC6bccd,Thalía,No Me Acuerdo,62,109991625,833269.886364,132,23wEWD21D4TPYiJugoXmYb,1725195,"[u'dance pop', u'latin', u'latin pop', u'mexic...",...,0.0873,0.1870,0.000000,0.836,7,217653,-4.247,0,0.772,1
3,00gpGR84M27moP7AFuqHIx,DJ A1,Bounce Out With That FT. YBN Nahmir,124,4816077,688011.000000,7,76STnzdGp9dpBe7nX7rQB2,11,[],...,0.1730,0.0426,0.000000,0.857,8,91011,-8.278,1,0.482,2
4,00TVmX9spquyg0UKuN57l3,Fall Out Boy,HOLD ME TIGHT OR DON’T,182,557997,557997.000000,1,4UXqAaa6dQYAk18Lv7PEgX,5729744,"[u'emo', u'modern rock', u'pop punk']",...,0.0500,0.0063,0.000003,0.581,6,210347,-4.324,0,0.847,1


In [7]:
y = data_df.cluster
X = data_df[['top.rank', 'total.plays', 'average.daily.plays'
                                                , 'days.in.chart', 'energy', 'liveness', 'tempo', 'speechiness'
                                                , 'acousticness', 'instrumentalness', 'danceability', 'key'
                                                , 'duration_ms' ,'loudness', 'mode', 'valence','popularity']]

In [8]:
X.head()

,top.rank,total.plays,average.daily.plays,days.in.chart,energy,liveness,tempo,speechiness,acousticness,instrumentalness,danceability,key,duration_ms,loudness,mode,valence,popularity
0,91,9809956,700711.142857,14,0.755,0.390,191.153,0.7330,0.0822,0.000000,0.353,1,193680,-6.276,0,0.437,83
1,61,11882167,848726.214286,14,0.625,0.314,167.911,0.5530,0.2870,0.000000,0.620,9,284856,-7.438,1,0.665,94
2,62,109991625,833269.886364,132,0.799,0.092,94.033,0.0873,0.1870,0.000000,0.836,7,217653,-4.247,0,0.772,78
3,124,4816077,688011.000000,7,0.560,0.153,94.949,0.1730,0.0426,0.000000,0.857,8,91011,-8.278,1,0.482,0
4,182,557997,557997.000000,1,0.919,0.107,95.991,0.0500,0.0063,0.000003,0.581,6,210347,-4.324,0,0.847,83


In [9]:
y.head()

0    1
1    1
2    1
3    2
4    1
Name: cluster, dtype: int64

In [10]:
# dividing X, y into train and test data 
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 0) 

In [11]:
dtree_model = DecisionTreeClassifier(max_depth = 3).fit(X_train, y_train) 
dtree_predictions = dtree_model.predict(X_test) 

In [12]:
# creating a confusion matrix 
cm = confusion_matrix(y_test, dtree_predictions) 

In [13]:
cm

array([[121,  11,   3,   4],
       [ 15, 147,   4,   1],
       [  1,   7,  45,   1],
       [  2,   8,   1,  41]])

In [14]:
accuracy_score(y_test, dtree_predictions)

0.85922330097087374

In [23]:
#optimise max depth parameter
for i in range(2,6):
    
    dtree_model = DecisionTreeClassifier(max_depth = i).fit(X_train, y_train) 
    dtree_predictions = dtree_model.predict(X_test)
    cm = confusion_matrix(y_test, dtree_predictions) 
    scores = cross_val_score(dtree_model,X,y,cv=10)
    print 'Max depth: %s' % i
    print cm
    print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))
    print

Max depth: 2
[[124  11   0   4]
 [ 16 150   0   1]
 [  7  46   0   1]
 [  2   9   0  41]]
Accuracy: 0.79 (+/- 0.05)

Max depth: 3
[[121  11   3   4]
 [ 15 147   4   1]
 [  1   7  45   1]
 [  2   8   1  41]]
Accuracy: 0.86 (+/- 0.06)

Max depth: 4
[[123  11   1   4]
 [ 16 149   1   1]
 [  5  10  38   1]
 [  2   8   1  41]]
Accuracy: 0.86 (+/- 0.07)

Max depth: 5
[[125  11   1   2]
 [ 16 150   0   1]
 [  3   9  41   1]
 [  2   8   1  41]]
Accuracy: 0.86 (+/- 0.06)



In [31]:
#since there is no improvement in accuracy at max_depth>3, use 3 for final model
dtree_model = DecisionTreeClassifier(max_depth = 3).fit(X_train, y_train) 
dtree_predictions = dtree_model.predict(X_test)

In [52]:
scores = cross_val_score(dtree_model,X,y,cv=10)
cm = confusion_matrix(y_test, dtree_predictions)

In [53]:
scores

array([ 0.89156627,  0.81325301,  0.85542169,  0.87951807,  0.81325301,
        0.90909091,  0.88343558,  0.8404908 ,  0.88957055,  0.84662577])

In [54]:
print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))
print cm

Accuracy: 0.86 (+/- 0.06)
[[121  11   3   4]
 [ 15 147   4   1]
 [  1   7  45   1]
 [  2   8   1  41]]


In [35]:
from sklearn import tree
import graphviz

In [49]:
dot_data = tree.export_graphviz(dtree_model,
                            feature_names=X.columns,
                            class_names=['1','2','3','4'],    
                            filled=True, rounded=True,  
                            special_characters=True) 
 

In [50]:
graph = graphviz.Source(dot_data)

In [51]:
#render the graph as a png file
!dot -Tpng tree.dot -o tree.png 